In [1]:
#import tflearn packages

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

#import other packages
import pandas as pd
import sklearn
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from skimage import measure
%matplotlib inline

import os
from os import getcwd
from os import listdir
from os.path import isfile, join, isdir

In [9]:
#randomly choose a subset of training images to make it run faster

#import images

def get_paths(foldNames):
  
    paths = dict.fromkeys(foldNames)

    for idx,g in enumerate(foldNames):
        fileNames = [f for f in listdir(join(trainPath,g)) if isfile(join(trainPath,g, f))]
        for i,f in enumerate(fileNames):
            fileNames[i] = join(trainPath,g,f)     
        paths[g] = fileNames
        
    return paths

trainPath = '../data/raw/train'
testPath = '../data/raw/test_stg1'
fish_classes = [f for f in listdir(trainPath) if isdir(join(trainPath, f))]
groupData = pd.DataFrame ({'group': fish_classes})
fish_paths = get_paths(fish_classes)

for k,v in fish_paths.items():
    print(k, v)

#label images by directory

#downsample images
def read_image(src):
    """Read and resize individual images"""
    im = io.imread(src)
    im = resize(im, (ROWS, COLS))
    return im

('SHARK', ['../data/raw/train/SHARK/img_00033.jpg', '../data/raw/train/SHARK/img_00072.jpg', '../data/raw/train/SHARK/img_00096.jpg', '../data/raw/train/SHARK/img_00235.jpg', '../data/raw/train/SHARK/img_00245.jpg', '../data/raw/train/SHARK/img_00247.jpg', '../data/raw/train/SHARK/img_00265.jpg', '../data/raw/train/SHARK/img_00365.jpg', '../data/raw/train/SHARK/img_00391.jpg', '../data/raw/train/SHARK/img_00409.jpg', '../data/raw/train/SHARK/img_00427.jpg', '../data/raw/train/SHARK/img_00478.jpg', '../data/raw/train/SHARK/img_00510.jpg', '../data/raw/train/SHARK/img_00560.jpg', '../data/raw/train/SHARK/img_00600.jpg', '../data/raw/train/SHARK/img_00685.jpg', '../data/raw/train/SHARK/img_00703.jpg', '../data/raw/train/SHARK/img_00736.jpg', '../data/raw/train/SHARK/img_00903.jpg', '../data/raw/train/SHARK/img_01061.jpg', '../data/raw/train/SHARK/img_01161.jpg', '../data/raw/train/SHARK/img_01182.jpg', '../data/raw/train/SHARK/img_01298.jpg', '../data/raw/train/SHARK/img_01307.jpg', '../d

In [ ]:
#build a simple CNN

In [ ]:
#test on subset of training images you didn't use; if it works well could do a full train:test and submit